# Sweep a option in QComponent, get Ansys HFSS driven-modal scatter matrix, impedance matrix, and admittance matrix.

# Prerequisite
You need to have a working local installation of Ansys

## 1. Perform the necessary imports and create a QDesign in Metal first.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr
from qiskit_metal.analyses.sweep_options.sweeping import Sweeping

## Create the design in Metal
 Set up a design of a given dimension.  Create a design by specifying the chip size and open Metal GUI.  Dimensions will be respected in the design rendering.  Note the chip design is centered at origin (0,0).

In [4]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

# Perform the necessary imports.
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround

In [6]:
# Add 2 transmons to the design.
options = dict(
    # Some options we want to modify from the deafults
    # (see below for defaults)
    pad_width='425 um',
    pocket_height='650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(a=dict(loc_W=+1, loc_H=+1),
                         b=dict(loc_W=-1, loc_H=+1, pad_height='30um'),
                         c=dict(loc_W=+1, loc_H=-1, pad_width='200um'),
                         d=dict(loc_W=-1, loc_H=-1, pad_height='50um')))

## Create 2 transmons
q1 = TransmonPocket(design,
                    'Q1',
                    options=dict(pos_x='+1.4mm',
                                 pos_y='0mm',
                                 orientation='90',
                                 **options))
q2 = TransmonPocket(design,
                    'Q2',
                    options=dict(pos_x='-0.6mm',
                                 pos_y='0mm',
                                 orientation='90',
                                 **options))

gui.rebuild()
gui.autoscale()

In [7]:
# Add 2 hangers consisting of capacitively coupled transmission lines.
TQ1 = CoupledLineTee(design,
                     'TQ1',
                     options=dict(pos_x='1mm',
                                  pos_y='3mm',
                                  coupling_length='200um'))
TQ2 = CoupledLineTee(design,
                     'TQ2',
                     options=dict(pos_x='-1mm',
                                  pos_y='3mm',
                                  coupling_length='200um'))

gui.rebuild()
gui.autoscale()

In [8]:
# Add 2 meandered CPWs connecting the transmons to the hangers.
ops = dict(fillet='90um')
design.overwrite_enabled = True

options1 = Dict(total_length='8mm',
                hfss_wire_bonds=True,
                pin_inputs=Dict(start_pin=Dict(component='TQ1',
                                               pin='second_end'),
                                end_pin=Dict(component='Q1', pin='a')),
                lead=Dict(start_straight='0.1mm'),
                **ops)

options2 = Dict(total_length='9mm',
                hfss_wire_bonds=True,
                pin_inputs=Dict(start_pin=Dict(component='TQ2',
                                               pin='second_end'),
                                end_pin=Dict(component='Q2', pin='a')),
                lead=Dict(start_straight='0.1mm'),
                **ops)

meanderQ1 = RouteMeander(design, 'meanderQ1', options=options1)
meanderQ2 = RouteMeander(design, 'meanderQ2', options=options2)

gui.rebuild()
gui.autoscale() 

In [9]:
# Add 2 open to grounds at the ends of the horizontal CPW.
otg1 = OpenToGround(design, 'otg1', options=dict(pos_x='3mm', pos_y='3mm'))
otg2 = OpenToGround(design,
                    'otg2',
                    options=dict(pos_x='-3mm', pos_y='3mm', orientation='180'))

gui.rebuild()
gui.autoscale()

# Add 3 straight CPWs that comprise the long horizontal CPW.

ops_oR = Dict(hfss_wire_bonds=True,
              pin_inputs=Dict(start_pin=Dict(component='TQ1', pin='prime_end'),
                              end_pin=Dict(component='otg1', pin='open')))
ops_mid = Dict(hfss_wire_bonds=True,
               pin_inputs=Dict(start_pin=Dict(component='TQ1',
                                              pin='prime_start'),
                               end_pin=Dict(component='TQ2', pin='prime_end')))
ops_oL = Dict(hfss_wire_bonds=True,
              pin_inputs=Dict(start_pin=Dict(component='TQ2',
                                             pin='prime_start'),
                              end_pin=Dict(component='otg2', pin='open')))

cpw_openRight = RouteStraight(design, 'cpw_openRight', options=ops_oR)
cpw_middle = RouteStraight(design, 'cpw_middle', options=ops_mid)
cpw_openLeft = RouteStraight(design, 'cpw_openLeft', options=ops_oL)

gui.rebuild()
gui.autoscale()

## 2. Render the qubit from Metal into the HangingResonators design in Ansys. <br>Must open Ansys HFSS at this point. 

**Wait for Ansys to fully open before proceeding.**<br> If necessary, also close any Ansys popup windows.

### Registration occurs automatically within the method "sweep_one_option_get_drivenmodal_solution_data". 

### Must have an open Ansys project and HFSS eigenmode design before executing render_design. Can do it manually or automate it. 

Manually: To create a new Ansys project, click "New", the leftmost icon right below the file menu. 

Automate: Execute the command <hfss.open_ansys()> in cell below only if Ansys is not already open.  Use the cells below to open the Ansys application.  

### The method "sweep_one_option_get_drivenmodal_solution_data" will connect to ansys, activate a HFSS design, add a setup, and insert frequency-sweep under the setup.

In [11]:
#Reference to Ansys hfss QRenderer
hfss = design.renderers.hfss

# This cell will open the Ansys application.  Run this cell, if Ansys, is not already open.
# A project will be opened using this command.
#hfss.open_ansys()

In [12]:
sweep = Sweeping(design)

In [13]:
# Set the buffer width at the edge of the design to be 0.5 mm 
# in both directions.

hfss.options['x_buffer_width_mm'] = 0.5
hfss.options['y_buffer_width_mm'] = 0.5

As a reminder, arguments for render_design() are given as: <br>

First parameter: List of components to render (empty list if rendering whole Metal design) <br><br>
Second parameter: List of pins (qcomp, pin) with open endcaps <br><br>
Third parameter: List of pins (qcomp, pin, impedance) to render as lumped ports <br><br>
Fourth parameter: List of junctions (qcomp, qgeometry_name, impedance, draw_ind)
       to render as lumped ports or as lumped port in parallel with a sheet inductance <br><br>
Fifth parameter: List of junctions (qcomp, qgeometry_name) to omit altogether during rendering<br><br>
Sixth parameter: Whether to render chip via box plus buffer or fixed chip size<br>


The size scatter square matrix, impedance square matrix, and admittance square matrix is determined by size of (3rd parameter) + (size of fourth parameter) of render_design.  

In [16]:
selection = [] # Means to render everything in qgeometry table.
open_pins = []

# Here, pin cpw_openRight_end and cpw_openLeft_end are converted into lumped ports,
#           each with an impedance of 50 Ohms. <br>
port_list = [('cpw_openRight', 'end', 50),
                      ('cpw_openLeft', 'end', 50)]
jj_to_port = []
# Neither of the junctions in Q1 or Q2 are rendered.
ignored_jjs = [('Q1', 'rect_jj'), ('Q2', 'rect_jj')]
box_plus_buffer = True

dm_render_args = Dict(selection=selection,
                      open_pins=open_pins,
                      port_list=port_list,
                      jj_to_port=jj_to_port,
                      ignored_jjs=ignored_jjs,
                      box_plus_buffer=box_plus_buffer)

In [17]:
# Pass a dict with arguments to be used add Setup to project.
# If you don't pass all the arguments, the default is determined by
# QHFSSRenderer's default_options.
# The name of setup will be "Sweep_dm_setup".
# If a setup named "Sweep_dm_setup" exists in the project, it will be deleted,
# and a new setup will be added with the arguments from setup_args.
hfss_dm_setup_args = Dict(freq_ghz=None,
                          max_delta_s=None,
                          max_passes=None,
                          min_passes=None,
                          min_converged=None,
                          pct_refinement=None,
                          basis_order=None)
# Example of updating the keys that we are interested in.
hfss_dm_setup_args = Dict(
    freq_ghz=4,
    max_delta_s=0.1,
    max_passes=9,
    min_passes=2,
)



Add a frequency sweep to a driven modal setup.<br>
From QHFSSRenderer.add_sweep doc_strings.  Please go to doc_strings to get the latest information.  

Args:<br>
    setup_name (str, optional): Name of driven modal simulation setup.
                            Defaults to "Setup".<br>
    start_ghz (float, optional): Starting frequency of sweep in GHz.
                            Defaults to 2.0.<br>
    stop_ghz (float, optional): Ending frequency of sweep in GHz.
                            Defaults to 8.0.<br>
    count (int, optional): Total number of frequencies.
                            Defaults to 101.<br>
    step_ghz (float, optional): Difference between adjacent
                            frequencies. Defaults to None.<br>
    name (str, optional): Name of sweep. Defaults to "Sweep".<br>
    type (str, optional): Type of sweep. Defaults to "Fast".<br>
    save_fields (bool, optional): Whether or not to save fields.
                        Defaults to False.<br>
                        
                        
From pyEPR.HfssSetup.insert_sweep(), please go to this method to get the latest documentation.   
You should provide either step_ghz or count when inserting an HFSS driven model freq sweep. Do not provide both or neither!

In [18]:
dm_add_sweep_args = Dict(name="Sweep_options__dm_sweep",
                         start_ghz=4.0,
                         stop_ghz=8.0,
                         count=2001,
                         type="Interpolating")

In [19]:
all_sweeps, return_code = sweep.sweep_one_option_get_drivenmodal_solution_data(
    meanderQ1.name,
    'total_length', ['9mm', '8mm', '7mm'],
    dm_render_args,
    setup_args=hfss_dm_setup_args,
    dm_add_sweep_args=dm_add_sweep_args,
    leave_last_design=True)

INFO 11:28AM [connect_project]: Connecting to Ansys Desktop API...
INFO 11:28AM [load_ansys_project]: 	Opened Ansys App
INFO 11:28AM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 11:28AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/priti.ashvin.shah/Documents/Ansoft/
	Project:   Project69
INFO 11:28AM [connect_design]: 	Opened active design
	Design:    Sweep_DrivenModal [Solution type: DrivenModal]
INFO 11:28AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 11:28AM [connect]: 	Connected to project "Project69" and design "Sweep_DrivenModal" 😀 

INFO 11:28AM [connect_design]: 	Opened active design
	Design:    Sweep_DrivenModal [Solution type: DrivenModal]
INFO 11:28AM [get_setup]: 	Opened setup `Sweep_dm_setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 11:30AM [get_setup]: 	Opened setup `Sweep_dm_setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 11:30AM [get_setup]: 	Opened setup `Sweep_dm_setup`  (<class 'pyEPR.ansys.HfssDMSe

In [24]:
# Each key corresponds to list passed to ['9mm', '8mm', '7mm']
if return_code == 0:
    print(all_sweeps.keys())


dict_keys(['9mm', '8mm', '7mm'])


In [26]:
# Each values use for the given option, has three matrixes:  
# scatter, impedance, and addmitance.
if return_code == 0:
    print(all_sweeps['9mm'].keys())

dict_keys(['s_matrix', 'y_matrix', 'z_matrix'])


In [27]:
# For example, just one group of solution data.
if return_code == 0:
    print(all_sweeps['9mm']['s_matrix'])

                      S11                 S21                 S12  \
4.000 -0.099621-0.027476j -0.277818+0.955059j -0.277818+0.955059j   
4.002 -0.099658-0.027416j -0.277207+0.955235j -0.277207+0.955235j   
4.004 -0.099695-0.027357j -0.276595+0.955410j -0.276595+0.955410j   
4.006 -0.099732-0.027297j -0.275983+0.955585j -0.275983+0.955585j   
4.008 -0.099769-0.027237j -0.275371+0.955759j -0.275371+0.955759j   
...                   ...                 ...                 ...   
7.992 -0.030066+0.056013j  0.837593+0.542584j  0.837593+0.542584j   
7.994 -0.029998+0.055988j  0.837945+0.542046j  0.837945+0.542046j   
7.996 -0.029931+0.055963j  0.838297+0.541508j  0.838297+0.541508j   
7.998 -0.029863+0.055938j  0.838649+0.540969j  0.838649+0.540969j   
8.000 -0.029796+0.055913j  0.839000+0.540431j  0.839000+0.540431j   

                      S22  
4.000 -0.098814-0.030247j  
4.002 -0.098853-0.030191j  
4.004 -0.098891-0.030135j  
4.006 -0.098929-0.030078j  
4.008 -0.098967-0.030022j  
...

In [ ]:
# In HFSS, after the last sweep, should the design be cleared?  The argument "leave_last_design" is true by default. 
# If you want to delete after last sweep, change argument to false.

leave_last_design = False

all_sweeps, return_code = sweep.sweep_one_option_get_drivenmodal_solution_data(
    meanderQ1.name,
    'total_length', ['9mm', '8mm', '7mm'],
    dm_render_args,
    setup_args=hfss_dm_setup_args,
    dm_add_sweep_args=dm_add_sweep_args,
    leave_last_design=leave_last_design)

In [ ]:
# Uncomment next line if you would like to close the gui
# gui.main_window.close()